<a href="https://colab.research.google.com/github/simpleParadox/PSYCO-576-Fall-2020/blob/master/PSYCO_576_ML_for_Psychology_DW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this first Data Workshop, we will look at some ways to inspect data and get feel for how the data is structured.


Goals and learning objectives:


1.   List item
2.   List item




We will use the following dataset:


Original Paper:



Variables: 

